In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import scanpy as sc
import sklearn
import sklearn.metrics

from dataset import PDDDataset
from models import PhenoProfiler
from utils import *

# define hyperparameters
model = PhenoProfiler().cuda()
model_path = "result/PhenoProfiler/last.pt"
save_path = "Fig3/BBBC036/PhenoProfiler_36/"

if not os.path.exists(save_path):
    os.makedirs(save_path)

def build_loaders_inference(batch_size):
    print("Building loaders")
    dataset = PDDDataset(image_path = "../dataset/bbbc036/images/",
               embedding_path = "../dataset/bbbc036/embedding/",
               CSV_path = "../dataset/bbbc036/profiling.csv")
    
    dataset = torch.utils.data.ConcatDataset([dataset])
    test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, drop_last=False)

    print("Finished building loaders")
    return test_loader

def get_image_embeddings(model_path, model, batch_size):
    test_loader = build_loaders_inference(batch_size)

    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.eval()

    print("Finished loading model")
    
    test_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(test_loader):
            image_features = model.image_encoder(batch["image"].cuda())
            image_embeddings = model.image_projection(image_features)
            test_image_embeddings.append(image_embeddings)
    
    return torch.cat(test_image_embeddings)

In [ ]:
# Load metadata of index data
meta = pd.read_csv(os.path.join(f"../dataset/bbbc036/profiling.csv"))

MATRIX_FILE = save_path+"/cos_efn128combinedcellsout_conv6a_1e-2_e30.csv"
REG_PARAM = 1e-2

meta["broad_sample"] = meta["Treatment"].str.split("@", expand=True)[0]

features = np.load(save_path+"PhenoProfiler_36train_36test.npy").T
print(features.shape)
meta

(119736, 672)


,Metadata_Plate,Metadata_Well,Metadata_Site,Plate_Map_Name,DNA,ER,RNA,AGP,Mito,broad_sample_Replicate,Treatment,Compound,Concentration,broad_sample
0,24277,a01,2,H-BIOA-004-3,24277/cdp2bioactives_a01_s2_w15e4541e6-dfcb-40...,24277/cdp2bioactives_a01_s2_w36b0ca5a6-63c8-44...,24277/cdp2bioactives_a01_s2_w26bec6edf-cec2-45...,24277/cdp2bioactives_a01_s2_w44bf0ab1d-a7a1-42...,24277/cdp2bioactives_a01_s2_w5d2a5a2a2-8548-40...,1,BRD-K18250272@3.02251611288227,BRD-K18250272,3.022516,BRD-K18250272
1,24277,a01,3,H-BIOA-004-3,24277/cdp2bioactives_a01_s3_w148f1c410-2d16-4c...,24277/cdp2bioactives_a01_s3_w32dc91611-633b-4a...,24277/cdp2bioactives_a01_s3_w29a7efc66-a795-4e...,24277/cdp2bioactives_a01_s3_w49abe4e30-4eed-41...,24277/cdp2bioactives_a01_s3_w5bf626b46-cc60-41...,1,BRD-K18250272@3.02251611288227,BRD-K18250272,3.022516,BRD-K18250272
2,24277,a01,4,H-BIOA-004-3,24277/cdp2bioactives_a01_s4_w1e9b39acd-2c91-4d...,24277/cdp2bioactives_a01_s4_w394c0f934-4bc1-47...,24277/cdp2bioactives_a01_s4_w2bc33bb34-f4a1-4d...,24277/cdp2bioactives_a01_s4_w41acd7309-157d-41...,24277/cdp2bioactives_a01_s4_w5d76fb6c3-8bf5-43...,1,BRD-K18250272@3.02251611288227,BRD-K18250272,3.022516,BRD-K18250272
3,24277,a01,5,H-BIOA-004-3,24277/cdp2bioactives_a01_s5_w196a63107-8e9d-41...,24277/cdp2bioactives_a01_s5_w3aa91cc5a-8c46-4a...,24277/cdp2bioactives_a01_s5_w23869d8be-1e44-42...,24277/cdp2bioactives_a01_s5_w4b254a0b2-f029-4b...,24277/cdp2bioactives_a01_s5_w5ae645261-7db0-40...,1,BRD-K18250272@3.02251611288227,BRD-K18250272,3.022516,BRD-K18250272
4,24277,a01,6,H-BIOA-004-3,24277/cdp2bioactives_a01_s6_w1dcad1406-ee25-41...,24277/cdp2bioactives_a01_s6_w3e008f0a8-1642-44...,24277/cdp2bioactives_a01_s6_w2fa2c1b40-ec9b-4d...,24277/cdp2bioactives_a01_s6_w4b328e14f-5559-4b...,24277/cdp2bioactives_a01_s6_w503cdcfe7-bb4d-4c...,1,BRD-K18250272@3.02251611288227,BRD-K18250272,3.022516,BRD-K18250272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119731,26239,p24,2,H-BIOA-007-3,26239/cdp2w9x2_p24_s2_w19d0e5649-1ff9-4eff-910...,26239/cdp2w9x2_p24_s2_w357a154e7-d806-4f59-a39...,26239/cdp2w9x2_p24_s2_w21c4f8fee-b646-405f-a14...,26239/cdp2w9x2_p24_s2_w4e5fb237d-ef45-47cb-8ee...,26239/cdp2w9x2_p24_s2_w5aefc843a-4326-4383-8b2...,7,BRD-K98521173@3.02607477854428,BRD-K98521173,3.026075,BRD-K98521173
119732,26239,p24,3,H-BIOA-007-3,26239/cdp2w9x2_p24_s3_w168798b74-6d99-4c3e-a0b...,26239/cdp2w9x2_p24_s3_w35d030cd5-26ec-46d9-9d8...,26239/cdp2w9x2_p24_s3_w2f04f803d-a2e7-426a-910...,26239/cdp2w9x2_p24_s3_w44ac9b4da-4491-4216-a2d...,26239/cdp2w9x2_p24_s3_w54274d296-8221-4f93-a95...,7,BRD-K98521173@3.02607477854428,BRD-K98521173,3.026075,BRD-K98521173
119733,26239,p24,4,H-BIOA-007-3,26239/cdp2w9x2_p24_s4_w1fe0f7930-4191-4c14-aae...,26239/cdp2w9x2_p24_s4_w392fbf20e-dfdb-4b63-886...,26239/cdp2w9x2_p24_s4_w26b118c50-f3ae-474a-891...,26239/cdp2w9x2_p24_s4_w42ac95a22-e62f-436a-b3f...,26239/cdp2w9x2_p24_s4_w5c8c2a645-6e20-4ab2-bfc...,7,BRD-K98521173@3.02607477854428,BRD-K98521173,3.026075,BRD-K98521173
119734,26239,p24,5,H-BIOA-007-3,26239/cdp2w9x2_p24_s5_w13f378fc8-7715-4edb-849...,26239/cdp2w9x2_p24_s5_w33f6ce905-df13-4813-9b9...,26239/cdp2w9x2_p24_s5_w225e645b8-5888-41f0-a5f...,26239/cdp2w9x2_p24_s5_w42ba3ef44-4d03-408d-a1f...,26239/cdp2w9x2_p24_s5_w5a2bc0932-34b3-477d-9df...,7,BRD-K98521173@3.02607477854428,BRD-K98521173,3.026075,BRD-K98521173


In [67]:
# 用序号的方式来指定 control Wells，

# 提取 Compound 列中值为 'DMSO' 的所有行号
dmso_indices = meta.index[meta['Treatment'] == 'NA@NA'].tolist()

# 打印结果
print(len(dmso_indices), dmso_indices[:10])

20148 [71, 72, 73, 74, 75, 76, 77, 78, 79, 80]


In [68]:
# 提取 meta 中的第一列 'Metadata_Plate'
metadata_plate = meta['Metadata_Plate'].values.reshape(-1, 1)

# 将 'Metadata_Plate' 列融合到 features 的第一列
combined_features = np.hstack((metadata_plate, features))

# 将 combined_features 转换为 DataFrame
combined_df = pd.DataFrame(combined_features, columns=['Metadata_Plate'] + [f'{i}' for i in range(1, features.shape[1]+1)])

# 打印结果的形状以确认
print(combined_df.shape)

# 只保留dmso_indices对应的行
filtered_combined_df = combined_df.iloc[dmso_indices]

# 打印结果的形状以确认
print(filtered_combined_df.shape)

(119736, 673)
(20148, 673)


In [69]:
# control well 对应的 特征
control_df = filtered_combined_df.groupby(["Metadata_Plate"]).mean().reset_index()
control_df

,Metadata_Plate,1,2,3,4,5,6,7,8,9,...,663,664,665,666,667,668,669,670,671,672
0,24277.0,1.503749,2.536142,2.831183,1.898062,1.113944,0.768451,1.959279,0.298006,1.241627,...,1.439696,0.962750,-0.078740,0.443283,1.999443,1.422643,1.869176,1.822343,0.833807,0.753516
1,24278.0,1.346064,2.584823,2.896558,1.990607,1.412027,0.878744,1.996764,0.329100,1.315463,...,1.409083,0.926790,0.102787,0.497505,2.010145,1.560972,1.925699,1.948685,0.800469,0.876064
2,24279.0,1.094893,2.534685,2.994968,1.922108,1.514683,0.988574,2.176054,0.303608,1.329729,...,1.474145,0.824279,0.256071,0.541311,1.918437,1.481312,2.061489,1.814424,0.747951,0.871537
3,24280.0,1.108433,2.415554,2.958619,2.115477,1.477212,0.958001,2.373423,0.284668,1.302327,...,1.361051,0.835090,0.345764,0.558762,1.755131,1.464937,1.975333,1.972411,0.739892,0.839073
4,24293.0,1.283059,2.452707,2.739062,2.046457,1.410774,0.944566,2.262570,0.305298,1.394304,...,1.434327,0.958618,0.266460,0.512521,1.994665,1.535301,1.743584,1.902788,0.770019,0.903316
5,24294.0,0.992572,2.255970,2.881122,2.052904,1.521417,0.989597,2.460357,0.255467,1.280644,...,1.525621,0.897168,0.738035,0.540581,1.721220,1.280910,1.841541,1.960143,0.753078,0.998756
6,24295.0,0.962338,2.228513,2.831851,2.074456,1.522179,0.994095,2.408873,0.260108,1.238171,...,1.379166,0.812775,0.641733,0.514413,1.703490,1.401477,1.789286,1.851843,0.748620,0.954851
7,24296.0,0.891744,2.157765,2.731691,1.491845,1.315653,0.899747,2.261124,0.261244,1.341962,...,1.488469,0.641740,0.459902,0.645896,1.701423,1.414382,2.674846,1.654766,0.726422,0.973407
8,24297.0,1.217619,2.406388,2.745625,1.467249,1.335222,0.885299,2.076637,0.339279,1.349880,...,1.433614,0.793931,0.151042,0.624988,1.990052,1.487589,2.487146,1.928425,0.722098,0.961880
9,24300.0,1.119970,2.431981,2.714449,1.267471,1.396491,0.941156,2.108450,0.288095,1.372213,...,1.512733,0.723128,0.222547,0.597221,1.978141,1.455501,2.517158,1.772608,0.732789,0.982011


In [ ]:
# 减去 control_df 中相同 Metadata_Plate 的行对应的后面 672 特征维度
def subtract_control_features(row, control_df):
    plate = row['Metadata_Plate']
    control_row = control_df[control_df['Metadata_Plate'] == plate]
    if not control_row.empty:
        row.iloc[1:] = row.iloc[1:] - control_row.iloc[0, 1:]
    return row

adjusted_combined_df = combined_df.apply(subtract_control_features, axis=1, control_df=control_df)
# adjusted_combined_df

In [ ]:
# 移除第一列 'Metadata_Plate'，然后变成 (66558, 672)
# adjusted_combined_df = adjusted_combined_df.drop(columns=['Metadata_Plate'])

# 保存为 .npy 文件
save_path = "Fig3/BBBC036/PhenoProfiler_test/PhenoProfiler_0.8.npy"
np.save(save_path, adjusted_combined_df.values)

# 打印结果的形状以确认
print(adjusted_combined_df.shape)

(119736, 673)
